# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

##**Step 1. Setup (2 Marks)**

(A) Writing/Creating the config.json file  (2 Marks)

### Installation

In [1]:
!pip install openai==0.28.0 tiktoken datasets session-info --quiet
!pip install scikit-learn
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


### Imports

Import all Python packages required to access the Azure Open AI API and to access datasets and create examples.

In [2]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.
import openai
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

### Authentication

**(A) Writing/Creating the config.json file (2 Marks)**

In [28]:
# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

#= RamNath's AzureAI Credentials

In [29]:
# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

Config file created successfully!


Reading the config.json file

In [30]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [31]:
creds = json.loads(data)

In [32]:
openai.api_key = creds["AZURE_OPENAI_KEY"]
openai.api_base = creds["AZURE_OPENAI_ENDPOINT"]
openai.api_type = creds["AZURE_OPENAI_APITYPE"]
openai.api_version = creds["AZURE_OPENAI_APIVERSION"]

In [8]:
chat_model_id = creds["CHATGPT_MODEL"]

### Utilities

Define a function for token counter to keep track of the completion window available in the prompt.

In [9]:
import tiktoken

def num_tokens_from_messages(messages):
    """
    Return the number of tokens used by a list of messages.
    Adapted from the OpenAI cookbook token counter.
    """
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with role and content tokens
    # Hence, messages look like: system or user or assistant {message}
    tokens_per_message = 3  # token1: role, token2: user/assistant/system, token3: content

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with 'assistant'

    return num_tokens

# Example usage
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather like today?"},
    {"role": "assistant", "content": "The weather is sunny and warm."}
]

print(num_tokens_from_messages(messages))


35


# Task: Sentiment Analysis

##**Step 2: Assemble Data (4 Marks)**

(A) Upload and Read csv File (2 Marks)

(B) Creating a new column named as "label" (target column) corresponding to the sentiments in the dataset (2 Marks)

Read the file

**(A) Upload and read csv file (2 Marks)**

In [10]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import io

# Create an upload button
upload_button = widgets.FileUpload()

# Function to handle file upload
def on_upload_change(change):
    for name, file_info in change['new'].items():
        # Read the file content into a pandas DataFrame
        df = pd.read_csv(io.BytesIO(file_info['content']))
        # Display the dataframe
        display(df.head())

# Attach the function to the upload button
upload_button.observe(on_upload_change, names='value')

# Display the upload button
display(upload_button)





FileUpload(value={}, description='Upload')

,id,review,sentiment
0,1,ExpressWay Logistics' commitment to transparen...,Positive
1,2,The tracking system implemented by ExpressWay ...,Positive
2,3,ExpressWay Logistics is a lifesaver when it co...,Positive
3,4,Expressway Logistics is the worst courier serv...,Negative
4,5,ExpressWay Logistics failed to meet my expecta...,Negative


In [11]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Read the uploaded CSV file
df = pd.read_csv("courier-service_reviews.csv")

# Display the first few rows of the dataframe
df.head()


Saving courier-service_reviews.csv to courier-service_reviews.csv


,id,review,sentiment
0,1,ExpressWay Logistics' commitment to transparen...,Positive
1,2,The tracking system implemented by ExpressWay ...,Positive
2,3,ExpressWay Logistics is a lifesaver when it co...,Positive
3,4,Expressway Logistics is the worst courier serv...,Negative
4,5,ExpressWay Logistics failed to meet my expecta...,Negative


**(B) Creating a new column named as "label" (target column) corresponding to the sentiments in the dataset (2 Marks)**

In [12]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Read the uploaded CSV file
df = pd.read_csv("courier-service_reviews.csv")

# Map sentiments to numeric labels
sentiment_to_label = {'Positive': 1, 'Negative': 0}
df['label'] = df['sentiment'].map(sentiment_to_label)

# Display the first few rows of the updated dataframe
df.head()

Saving courier-service_reviews.csv to courier-service_reviews (1).csv


,id,review,sentiment,label
0,1,ExpressWay Logistics' commitment to transparen...,Positive,1
1,2,The tracking system implemented by ExpressWay ...,Positive,1
2,3,ExpressWay Logistics is a lifesaver when it co...,Positive,1
3,4,Expressway Logistics is the worst courier serv...,Negative,0
4,5,ExpressWay Logistics failed to meet my expecta...,Negative,0


Split the data into two segments (use split_ratio of 0.2) - one segment (80%) that gives us a pool to draw few-shot examples from and another segment (20%) that gives us a pool of gold examples.

In [13]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Read the uploaded CSV file
df = pd.read_csv("courier-service_reviews.csv")

# Map sentiments to numeric labels
sentiment_to_label = {'Positive': 1, 'Negative': 0}
df['label'] = df['sentiment'].map(sentiment_to_label)

# Split the data into two segments with a split ratio of 0.2
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Display the first few rows of the training and testing dataframes
print("Training Data (80%):")
print(train_df.head())

print("\nTesting Data (20%):")
print(test_df.head())


Saving courier-service_reviews.csv to courier-service_reviews (2).csv
Training Data (80%):
    id                                             review sentiment  label
78  79  The packaging used by ExpressWay Logistics for...  Negative      0
47  48  When planning a high-profile corporate event, ...  Positive      1
0    1  ExpressWay Logistics' commitment to transparen...  Positive      1
12  13  The delivery person was too rude and frustrate...  Negative      0
42  43  I encountered numerous issues with ExpressWay ...  Negative      0

Testing Data (20%):
      id                                             review sentiment  label
55    56  ExpressWay Logistics offers a convenient onlin...  Negative      0
40    41  ExpressWay Logistics' pricing may seem attract...  Positive      1
19    20  I was deeply disappointed by the behavior of t...  Negative      0
31    32  The packaging provided by ExpressWay Logistics...  Positive      1
115  116  ExpressWay Logistics has been my go-to cour

Select the correct columns for further analysis which should exclude the target column.

In [14]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Read the uploaded CSV file
df = pd.read_csv("courier-service_reviews.csv")

# Map sentiments to numeric labels
sentiment_to_label = {'Positive': 1, 'Negative': 0}
df['label'] = df['sentiment'].map(sentiment_to_label)

# Split the data into two segments with a split ratio of 0.2
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Select the columns for further analysis, excluding the target column 'label'
train_features = train_df.drop(columns=['label'])
test_features = test_df.drop(columns=['label'])

# Display the first few rows of the training and testing features
print("Training Features (80%):")
print(train_features.head())

print("\nTesting Features (20%):")
print(test_features.head())


Saving courier-service_reviews.csv to courier-service_reviews (3).csv
Training Features (80%):
    id                                             review sentiment
78  79  The packaging used by ExpressWay Logistics for...  Negative
47  48  When planning a high-profile corporate event, ...  Positive
0    1  ExpressWay Logistics' commitment to transparen...  Positive
12  13  The delivery person was too rude and frustrate...  Negative
42  43  I encountered numerous issues with ExpressWay ...  Negative

Testing Features (20%):
      id                                             review sentiment
55    56  ExpressWay Logistics offers a convenient onlin...  Negative
40    41  ExpressWay Logistics' pricing may seem attract...  Positive
19    20  I was deeply disappointed by the behavior of t...  Negative
31    32  The packaging provided by ExpressWay Logistics...  Positive
115  116  ExpressWay Logistics has been my go-to courier...  Negative


Create gold examples and select a random sample (depends on the learner based on the session runtime - example:21) of rows from the gold examples dataframe(cs_gold_examples_df).

In [15]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Read the uploaded CSV file
df = pd.read_csv("courier-service_reviews.csv")

# Map sentiments to numeric labels
sentiment_to_label = {'Positive': 1, 'Negative': 0}
df['label'] = df['sentiment'].map(sentiment_to_label)

# Split the data into two segments with a split ratio of 0.2
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Select the columns for further analysis, excluding the target column 'label'
train_features = train_df.drop(columns=['label'])
test_features = test_df.drop(columns=['label'])

# Create gold examples dataframe from the test segment
cs_gold_examples_df = test_df

# Select a random sample of rows from the gold examples dataframe
sample_size = 21  # Specify the sample size
gold_sample_df = cs_gold_examples_df.sample(n=sample_size, random_state=42)

# Display the first few rows of the gold sample dataframe
print("Gold Sample DataFrame:")
print(gold_sample_df.head())


Saving courier-service_reviews.csv to courier-service_reviews (4).csv
Gold Sample DataFrame:
    id                                             review sentiment  label
81  82  The delivery executive assigned by ExpressWay ...  Positive      1
4    5  ExpressWay Logistics failed to meet my expecta...  Negative      0
26  27  ExpressWay Logistics' incompetence resulted in...  Negative      0
10  11  Kudos to ExpressWay Logistics for their outsta...  Positive      1
55  56  ExpressWay Logistics offers a convenient onlin...  Negative      0


To select gold examples for this session, sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

##**Step 3: Derive Prompt (14 Marks)**

(A) Write Zero Shot Prompt (5 Marks)

(B) Write Few Shot Prompt (5 Marks)

(C) Print Create Examples (2 Marks)

(D) Print Few shot Prompt (2 Marks)

#### Create prompts

InvalidRequestError: Resource not found


**(A) Write Zero Shot Prompt (5 Marks)**

In [24]:
import openai

openai.api_key = '3c6bca5125454957999ec0c2502cda96'
openai.api_base = 'https://openairesource02.openai.azure.com/'
openai.api_type = 'azure'
openai.api_version = '2023-03-15-preview'

response = openai.Completion.create(
    engine='igpt-35-turbo',  # Use the correct deployment name
    prompt="Say this is a test",
    max_tokens=5
)

print(response.choices[0].text.strip())


string";
  const int


In [35]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
with open('config.json') as config_file:  # Corrected file path
    config = json.load(config_file)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

# Define the zero-shot prompt
prompt = """
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=prompt,
        max_tokens=50
    )
    # Print the response
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
## Solution

1. Positive
2. Positive
3. Positive
4. Negative
5. Negative

The data in the review is quite straightforward lending to clear sentiments. ExpressWay is well-spoken of in the Positive feedback, while in


In [37]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
with open('config.json') as config_file:  # Corrected file path
    config = json.load(config_file)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

# Define the zero-shot prompt system message
zero_shot_system_message = """
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

# Create the prompt for the completion
zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=zero_shot_system_message,
        max_tokens=50
    )
    # Print the response
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
## Solution

### Import Libraries
First let's import the necessary libraries.

### Dataset
The dataset has been provided by the company
### Data Processing and Cleaning
The data processing and Cleaning is not required as the data has already been clean. We


**(B) Write Few Shot Prompt (5 Marks)**

For the few-shot prompt, there is no change in the system message compared with the zero-shot prompt. However, we augment this system message with few shot examples.  

In [39]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
with open('config.json') as config_file:  # Corrected file path
    config = json.load(config_file)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

# Define the system message with a few-shot example
few_shot_system_message = """
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews:

Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative

Please classify the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
"""

# ExpressWay Logistics' commitment to transparency is impressive. -> Positive
# The tracking system implemented by ExpressWay is user-friendly and reliable. -> Positive
# ExpressWay Logistics is a lifesaver when it comes to timely deliveries. -> Positive


Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order.

Let us now look at how we can assemble examples to go along with this few-shot system message and compose a few-shot prompt.

###Define "create_examples" function

In [40]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
with open('config.json') as config_file:  # Corrected file path
    config = json.load(config_file)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the examples to use in the few-shot prompt
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative")
]

# Create the few-shot system message
few_shot_system_message = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{create_examples(examples)}

Please classify the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
'''


# I will use a simple approach for this. I will look for the words that indicate postive and negative sentiments

# Positive words: outstanding, impressive, user-friendly, reliable, lifesaver, timely deliveries
# Negative words: worst


Use the above create_examples function to create examples for few shot prompt

In [42]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:  # Corrected file path
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the examples to use in the few-shot prompt
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative"),
    ("Their customer support is impressive and always helpful.", "Positive"),
    ("The delivery was delayed and the package was damaged.", "Negative"),
    ("The new software update is user-friendly and reliable.", "Positive"),
    ("This is the worst experience I've ever had with any company.", "Negative")
]

# Create the few-shot system message
few_shot_system_message = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{create_examples(examples)}

Please classify the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

print("Few-shot system message created successfully!")

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print("API response received successfully!")
    print("Sentiment Classification Results:")
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Few-shot system message created successfully!
API response received successfully!
Sentiment Classification Results:
## Solution

1. Positive
2. Positive
3. Positive
4. Negative
5. Negative<|im_end|>


**(C) Print Created Examples (2 Marks)**

In [43]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:  # Corrected file path
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the examples to use in the few-shot prompt
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative"),
    ("Their customer support is impressive and always helpful.", "Positive"),
    ("The delivery was delayed and the package was damaged.", "Negative"),
    ("The new software update is user-friendly and reliable.", "Positive"),
    ("This is the worst experience I've ever had with any company.", "Negative")
]

# Create the few-shot system message
created_examples = create_examples(examples)
few_shot_system_message = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{created_examples}

Please classify the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

print("Created Examples:")
print(created_examples)

print("Few-shot system message created successfully!")

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print("API response received successfully!")
    print("Sentiment Classification Results:")
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Created Examples:
Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressive and always helpful." -> Positive
4. "The delivery was delayed and the package was damaged." -> Negative
5. "The new software update is user-friendly and reliable." -> Positive
6. "This is the worst experience I've ever had with any company." -> Negative

Few-shot system message created successfully!
API response received successfully!
Sentiment Classification Results:
"""
print('1. Positive')
print('2. Positive')
print('3. Positive')
print('4. Negative')
print('5. Negative')<|im_sep|>


2. Define "create_prompt" function

In [44]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:  # Corrected file path
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the create_prompt function
def create_prompt(examples, reviews_to_classify):
    examples_str = create_examples(examples)
    prompt = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{examples_str}

Please classify the following reviews:
"""
    for i, review in enumerate(reviews_to_classify, 1):
        prompt += f"{i}. \"{review}\"\n"

    prompt += "\nPlease provide the sentiment for each review in the format:\n"
    for i in range(1, len(reviews_to_classify) + 1):
        prompt += f"{i}. [Sentiment]\n"

    return prompt

# Define the examples to use in the few-shot prompt
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative"),
    ("Their customer support is impressive and always helpful.", "Positive"),
    ("The delivery was delayed and the package was damaged.", "Negative"),
    ("The new software update is user-friendly and reliable.", "Positive"),
    ("This is the worst experience I've ever had with any company.", "Negative")
]

# Define the reviews to classify
reviews_to_classify = [
    "ExpressWay Logistics' commitment to transparency is impressive.",
    "The tracking system implemented by ExpressWay is user-friendly and reliable.",
    "ExpressWay Logistics is a lifesaver when it comes to timely deliveries.",
    "Expressway Logistics is the worst courier service I've ever used.",
    "ExpressWay Logistics failed to meet my expectations."
]

# Create the few-shot system message
few_shot_system_message = create_prompt(examples, reviews_to_classify)

print("Created Examples:")
print(create_examples(examples))

print("Few-shot system message created successfully!")
print(few_shot_system_message)

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print("API response received successfully!")
    print("Sentiment Classification Results:")
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Created Examples:
Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressive and always helpful." -> Positive
4. "The delivery was delayed and the package was damaged." -> Negative
5. "The new software update is user-friendly and reliable." -> Positive
6. "This is the worst experience I've ever had with any company." -> Negative

Few-shot system message created successfully!

I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressi

Use the above create_prompt function to create few_shot_prompt

In [45]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:  # Corrected file path
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the create_prompt function
def create_prompt(examples, reviews_to_classify):
    examples_str = create_examples(examples)
    prompt = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{examples_str}

Please classify the following reviews:
"""
    for i, review in enumerate(reviews_to_classify, 1):
        prompt += f"{i}. \"{review}\"\n"

    prompt += "\nPlease provide the sentiment for each review in the format:\n"
    for i in range(1, len(reviews_to_classify) + 1):
        prompt += f"{i}. [Sentiment]\n"

    return prompt

# Define the examples to use in the few-shot prompt
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative"),
    ("Their customer support is impressive and always helpful.", "Positive"),
    ("The delivery was delayed and the package was damaged.", "Negative"),
    ("The new software update is user-friendly and reliable.", "Positive"),
    ("This is the worst experience I've ever had with any company.", "Negative")
]

# Define the reviews to classify
reviews_to_classify = [
    "ExpressWay Logistics' commitment to transparency is impressive.",
    "The tracking system implemented by ExpressWay is user-friendly and reliable.",
    "ExpressWay Logistics is a lifesaver when it comes to timely deliveries.",
    "Expressway Logistics is the worst courier service I've ever used.",
    "ExpressWay Logistics failed to meet my expectations."
]

# Create the few-shot system message using create_prompt
few_shot_system_message = create_prompt(examples, reviews_to_classify)

print("Created Examples:")
print(create_examples(examples))

print("Few-shot system message created successfully!")
print(few_shot_system_message)

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print("API response received successfully!")
    print("Sentiment Classification Results:")
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Created Examples:
Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressive and always helpful." -> Positive
4. "The delivery was delayed and the package was damaged." -> Negative
5. "The new software update is user-friendly and reliable." -> Positive
6. "This is the worst experience I've ever had with any company." -> Negative

Few-shot system message created successfully!

I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressi

**(D) Print the Few Shot Prompt (2 Marks)**

In [46]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:  # Corrected file path
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the create_prompt function
def create_prompt(examples, reviews_to_classify):
    examples_str = create_examples(examples)
    prompt = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{examples_str}

Please classify the following reviews:
"""
    for i, review in enumerate(reviews_to_classify, 1):
        prompt += f"{i}. \"{review}\"\n"

    prompt += "\nPlease provide the sentiment for each review in the format:\n"
    for i in range(1, len(reviews_to_classify) + 1):
        prompt += f"{i}. [Sentiment]\n"

    return prompt

# Define the examples to use in the few-shot prompt
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative"),
    ("Their customer support is impressive and always helpful.", "Positive"),
    ("The delivery was delayed and the package was damaged.", "Negative"),
    ("The new software update is user-friendly and reliable.", "Positive"),
    ("This is the worst experience I've ever had with any company.", "Negative")
]

# Define the reviews to classify
reviews_to_classify = [
    "ExpressWay Logistics' commitment to transparency is impressive.",
    "The tracking system implemented by ExpressWay is user-friendly and reliable.",
    "ExpressWay Logistics is a lifesaver when it comes to timely deliveries.",
    "Expressway Logistics is the worst courier service I've ever used.",
    "ExpressWay Logistics failed to meet my expectations."
]

# Create the few-shot system message using create_prompt
few_shot_system_message = create_prompt(examples, reviews_to_classify)

print("Created Examples:")
print(create_examples(examples))

print("Few-shot system message created successfully!")
print("Few-Shot Prompt:")
print(few_shot_system_message)

# Use the OpenAI API to get the completion
try:
    response = openai.Completion.create(
        engine=config['CHATGPT_MODEL'],  # Use the model defined in the config
        prompt=few_shot_system_message,
        max_tokens=50,
        temperature=0.7
    )
    # Print the response
    print("API response received successfully!")
    print("Sentiment Classification Results:")
    print(response.choices[0].text.strip())
except openai.error.InvalidRequestError as e:
    print(f"InvalidRequestError: {e}")
    print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
except openai.error.AuthenticationError as e:
    print(f"AuthenticationError: {e}")
except openai.error.APIConnectionError as e:
    print(f"APIConnectionError: {e}")
except openai.error.OpenAIError as e:
    print(f"OpenAIError: {e}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Created Examples:
Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressive and always helpful." -> Positive
4. "The delivery was delayed and the package was damaged." -> Negative
5. "The new software update is user-friendly and reliable." -> Positive
6. "This is the worst experience I've ever had with any company." -> Negative

Few-shot system message created successfully!
Few-Shot Prompt:

I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer su

##**Step 4: Evaluate prompts (10 Marks)**

(A) Evaluate Zero Shot Prompt (3 Marks)

(B) Evaluate Few Shot Prompt (3 marks)

(C) Calculate Mean and Standard Deviation for Zero Shot and Few Shot (4 Marks)

### Define Evaluation scorer

Zero-Shot Prompt Results:
['"""', '', '1. Positive', '2. Positive', '3. Positive', '4. Negative', '5. Negative<|im_end|>']
Mean Accuracy: 0.2, Standard Deviation: 0.0

Few-Shot Prompt Results:
["'''", 'print("1. Positive")', 'print("2. Positive")', 'print("3. Positive")', 'print("4. Negative")', 'print("5. Negative")', '', '# Output', "'''", '1. Positive', '2. Positive', '3. Positive', '4. Negative']
Mean Accuracy: 0.0, Standard Deviation: 0.0


**(A) Evaluate zero shot prompt (3 Marks)**

In [48]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:  # Corrected file path
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Zero-shot system message
zero_shot_system_message = """
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

# Function to get sentiment classification from OpenAI
def get_sentiment_classification(prompt):
    try:
        response = openai.Completion.create(
            engine=config['CHATGPT_MODEL'],
            prompt=prompt,
            max_tokens=50,
            temperature=0.7
        )
        return response.choices[0].text.strip().split("\n")
    except openai.error.OpenAIError as e:
        print(f"OpenAIError: {e}")
        return []

# Evaluate zero-shot prompt
zero_shot_results = get_sentiment_classification(zero_shot_system_message)

print("Zero-Shot Prompt Results:")
print(zero_shot_results)


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Zero-Shot Prompt Results:
['## Solution', '1. Positive', '2. Positive', '3. Positive', '4. Negative', '5. Negative', '', '## Explenation', '1. "ExpressWay Logistics\' commitment to transparency is impressive." The statement mentions the word \'impressive\'']


**(B) Evaluate few shot prompt (3 Marks)**

In [59]:
import json
import openai

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"  # Ensure this is the correct model deployment name
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the create_prompt function
def create_prompt(examples, reviews_to_classify):
    examples_str = create_examples(examples)
    prompt = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{examples_str}

Please classify the following reviews:
"""
    for i, review in enumerate(reviews_to_classify, 1):
        prompt += f"{i}. \"{review}\"\n"

    prompt += "\nPlease provide the sentiment for each review in the format:\n"
    for i in range(1, len(reviews_to_classify) + 1):
        prompt += f"{i}. [Sentiment]\n"

    return prompt

# Few-shot examples
examples = [
    ("The service at this restaurant is outstanding.", "Positive"),
    ("I am very disappointed with the quality of the product.", "Negative"),
    ("Their customer support is impressive and always helpful.", "Positive"),
    ("The delivery was delayed and the package was damaged.", "Negative"),
    ("The new software update is user-friendly and reliable.", "Positive"),
    ("This is the worst experience I've ever had with any company.", "Negative")
]

# Reviews to classify
reviews_to_classify = [
    "ExpressWay Logistics' commitment to transparency is impressive.",
    "The tracking system implemented by ExpressWay is user-friendly and reliable.",
    "ExpressWay Logistics is a lifesaver when it comes to timely deliveries.",
    "Expressway Logistics is the worst courier service I've ever used.",
    "ExpressWay Logistics failed to meet my expectations."
]

# Create the few-shot system message using create_prompt
few_shot_system_message = create_prompt(examples, reviews_to_classify)

print("Created Examples:")
print(create_examples(examples))

print("Few-shot system message created successfully!")
print("Few-Shot Prompt:")
print(few_shot_system_message)

# Function to get sentiment classification from OpenAI
def get_sentiment_classification(prompt):
    try:
        response = openai.Completion.create(
            engine=config['CHATGPT_MODEL'],
            prompt=prompt,
            max_tokens=50,
            temperature=0.7
        )
        return response.choices[0].text.strip().split("\n")
    except openai.error.InvalidRequestError as e:
        print(f"InvalidRequestError: {e}")
        print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
    except openai.error.AuthenticationError as e:
        print(f"AuthenticationError: {e}")
    except openai.error.APIConnectionError as e:
        print(f"APIConnectionError: {e}")
    except openai.error.OpenAIError as e:
        print(f"OpenAIError: {e}")
    return []

# Evaluate few-shot prompt
few_shot_results = get_sentiment_classification(few_shot_system_message)

print("Few-Shot Prompt Results:")
print(few_shot_results)


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Created Examples:
Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer support is impressive and always helpful." -> Positive
4. "The delivery was delayed and the package was damaged." -> Negative
5. "The new software update is user-friendly and reliable." -> Positive
6. "This is the worst experience I've ever had with any company." -> Negative

Few-shot system message created successfully!
Few-Shot Prompt:

I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

Examples:
1. "The service at this restaurant is outstanding." -> Positive
2. "I am very disappointed with the quality of the product." -> Negative
3. "Their customer su

 However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [60]:
import json
import openai
import numpy as np
from sklearn.metrics import f1_score

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"  # Ensure this is the correct model deployment name
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the create_prompt function
def create_prompt(examples, reviews_to_classify):
    examples_str = create_examples(examples)
    prompt = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{examples_str}

Please classify the following reviews:
"""
    for i, review in enumerate(reviews_to_classify, 1):
        prompt += f"{i}. \"{review}\"\n"

    prompt += "\nPlease provide the sentiment for each review in the format:\n"
    for i in range(1, len(reviews_to_classify) + 1):
        prompt += f"{i}. [Sentiment]\n"

    return prompt

# Function to get sentiment classification from OpenAI
def get_sentiment_classification(prompt):
    try:
        response = openai.Completion.create(
            engine=config['CHATGPT_MODEL'],
            prompt=prompt,
            max_tokens=50,
            temperature=0.7
        )
        return [line.split(" ")[-1].strip("[]") for line in response.choices[0].text.strip().split("\n") if line]
    except openai.error.InvalidRequestError as e:
        print(f"InvalidRequestError: {e}")
        print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
    except openai.error.AuthenticationError as e:
        print(f"AuthenticationError: {e}")
    except openai.error.APIConnectionError as e:
        print(f"APIConnectionError: {e}")
    except openai.error.OpenAIError as e:
        print(f"OpenAIError: {e}")
    return []

# Few-shot examples
example_sets = [
    [
        ("The service at this restaurant is outstanding.", "Positive"),
        ("I am very disappointed with the quality of the product.", "Negative"),
        ("Their customer support is impressive and always helpful.", "Positive"),
        ("The delivery was delayed and the package was damaged.", "Negative"),
        ("The new software update is user-friendly and reliable.", "Positive"),
        ("This is the worst experience I've ever had with any company.", "Negative")
    ],
    [
        ("The hotel staff were incredibly friendly.", "Positive"),
        ("The food was cold and unappetizing.", "Negative"),
        ("I love the new features in this update.", "Positive"),
        ("My order was incorrect and had to be sent back.", "Negative"),
        ("The packaging was excellent and very secure.", "Positive"),
        ("Customer service was very rude and unhelpful.", "Negative")
    ],
    [
        ("This is by far the best product I've used.", "Positive"),
        ("The installation process was very complicated.", "Negative"),
        ("Their attention to detail is commendable.", "Positive"),
        ("The item arrived broken and unusable.", "Negative"),
        ("Fantastic quality and speedy delivery.", "Positive"),
        ("I will never buy from this company again.", "Negative")
    ],
    [
        ("The app is very intuitive and easy to use.", "Positive"),
        ("The website crashed multiple times while ordering.", "Negative"),
        ("Great value for the price.", "Positive"),
        ("The staff ignored us for over an hour.", "Negative"),
        ("Highly recommended to all my friends.", "Positive"),
        ("The product did not match the description.", "Negative")
    ],
    [
        ("The best experience I've had in a long time.", "Positive"),
        ("The worst customer service I've encountered.", "Negative"),
        ("Excellent craftsmanship and attention to detail.", "Positive"),
        ("The product was defective and had to be returned.", "Negative"),
        ("Very happy with my purchase!", "Positive"),
        ("I was disappointed with the overall experience.", "Negative")
    ]
]

# Reviews to classify
reviews_to_classify = [
    "ExpressWay Logistics' commitment to transparency is impressive.",
    "The tracking system implemented by ExpressWay is user-friendly and reliable.",
    "ExpressWay Logistics is a lifesaver when it comes to timely deliveries.",
    "Expressway Logistics is the worst courier service I've ever used.",
    "ExpressWay Logistics failed to meet my expectations."
]

# Expected sentiments
expected_sentiments = ["Positive", "Positive", "Positive", "Negative", "Negative"]

# Evaluate few-shot prompts and calculate F1 scores
f1_scores = []

for examples in example_sets:
    few_shot_system_message = create_prompt(examples, reviews_to_classify)
    few_shot_results = get_sentiment_classification(few_shot_system_message)

    if len(few_shot_results) == len(expected_sentiments):
        f1 = f1_score(expected_sentiments, few_shot_results, average='weighted')
        f1_scores.append(f1)
        print(f"Few-Shot Prompt Results for example set: {examples}")
        print(f"Results: {few_shot_results}")
        print(f"F1 Score: {f1}")
    else:
        print(f"Failed to get valid results for example set: {examples}")
        print(f"Results: {few_shot_results}")

# Calculate mean and standard deviation of F1 scores
few_shot_mean = np.mean(f1_scores)
few_shot_std = np.std(f1_scores)

print("\nFew-Shot Evaluation Summary:")
print(f"Mean F1 Score: {few_shot_mean}")
print(f"Standard Deviation of F1 Scores: {few_shot_std}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Failed to get valid results for example set: [('The service at this restaurant is outstanding.', 'Positive'), ('I am very disappointed with the quality of the product.', 'Negative'), ('Their customer support is impressive and always helpful.', 'Positive'), ('The delivery was delayed and the package was damaged.', 'Negative'), ('The new software update is user-friendly and reliable.', 'Positive'), ("This is the worst experience I've ever had with any company.", 'Negative')]
Results: ['<|im_end|>']
Failed to get valid results for example set: [('The hotel staff were incredibly friendly.', 'Positive'), ('The food was cold and unappetizing.', 'Negative'), ('I love the new features in this update.', 'Positive'), ('My order was incorrect and had to be sent back.', 'Negative'), ('The packaging was excellent and very secure.', 'Positive'), ('Customer service was very rude and unhelpful

**(C) Calculate Mean and Standard Deviation for Zero Shot and Few Shot (4 Marks)**

Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [68]:
import json
import openai
import numpy as np
from sklearn.metrics import f1_score

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"  # Ensure this is the correct model deployment name
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Define the create_examples function
def create_examples(examples):
    example_str = "Examples:\n"
    for i, (review, sentiment) in enumerate(examples, 1):
        example_str += f"{i}. \"{review}\" -> {sentiment}\n"
    return example_str

# Define the create_prompt function
def create_prompt(examples, reviews_to_classify):
    examples_str = create_examples(examples)
    prompt = f"""
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

{examples_str}

Please classify the following reviews:
"""
    for i, review in enumerate(reviews_to_classify, 1):
        prompt += f"{i}. \"{review}\"\n"

    prompt += "\nPlease provide the sentiment for each review in the format:\n"
    for i in range(1, len(reviews_to_classify) + 1):
        prompt += f"{i}. [Sentiment]\n"

    return prompt

# Function to get sentiment classification from OpenAI
def get_sentiment_classification(prompt):
    print("Sending Prompt to OpenAI API:\n", prompt)  # Debug: Print the prompt
    try:
        response = openai.Completion.create(
            engine=config['CHATGPT_MODEL'],
            prompt=prompt,
            max_tokens=100,
            temperature=0.7
        )
        print("API Response:\n", response)  # Debug: Print the API response
        # Process the response to get only the sentiment labels
        results = [line.split(" ")[-1].strip("[]") for line in response.choices[0].text.strip().split("\n") if line]
        # Filter valid sentiments
        valid_sentiments = {'Positive', 'Negative'}
        return [result for result in results if result in valid_sentiments]
    except openai.error.InvalidRequestError as e:
        print(f"InvalidRequestError: {e}")
        print("Please ensure the deployment name is correct and the deployment exists in your Azure OpenAI resource.")
    except openai.error.AuthenticationError as e:
        print(f"AuthenticationError: {e}")
    except openai.error.APIConnectionError as e:
        print(f"APIConnectionError: {e}")
    except openai.error.OpenAIError as e:
        print(f"OpenAIError: {e}")
    return []

# Few-shot examples
example_sets = [
    [
        ("The service at this restaurant is outstanding.", "Positive"),
        ("I am very disappointed with the quality of the product.", "Negative"),
        ("Their customer support is impressive and always helpful.", "Positive"),
        ("The delivery was delayed and the package was damaged.", "Negative"),
        ("The new software update is user-friendly and reliable.", "Positive"),
        ("This is the worst experience I've ever had with any company.", "Negative")
    ],
    [
        ("The hotel staff were incredibly friendly.", "Positive"),
        ("The food was cold and unappetizing.", "Negative"),
        ("I love the new features in this update.", "Positive"),
        ("My order was incorrect and had to be sent back.", "Negative"),
        ("The packaging was excellent and very secure.", "Positive"),
        ("Customer service was very rude and unhelpful.", "Negative")
    ],
    [
        ("This is by far the best product I've used.", "Positive"),
        ("The installation process was very complicated.", "Negative"),
        ("Their attention to detail is commendable.", "Positive"),
        ("The item arrived broken and unusable.", "Negative"),
        ("Fantastic quality and speedy delivery.", "Positive"),
        ("I will never buy from this company again.", "Negative")
    ],
    [
        ("The app is very intuitive and easy to use.", "Positive"),
        ("The website crashed multiple times while ordering.", "Negative"),
        ("Great value for the price.", "Positive"),
        ("The staff ignored us for over an hour.", "Negative"),
        ("Highly recommended to all my friends.", "Positive"),
        ("The product did not match the description.", "Negative")
    ],
    [
        ("The best experience I've had in a long time.", "Positive"),
        ("The worst customer service I've encountered.", "Negative"),
        ("Excellent craftsmanship and attention to detail.", "Positive"),
        ("The product was defective and had to be returned.", "Negative"),
        ("Very happy with my purchase!", "Positive"),
        ("I was disappointed with the overall experience.", "Negative")
    ]
]

# Reviews to classify
reviews_to_classify = [
    "ExpressWay Logistics' commitment to transparency is impressive.",
    "The tracking system implemented by ExpressWay is user-friendly and reliable.",
    "ExpressWay Logistics is a lifesaver when it comes to timely deliveries.",
    "Expressway Logistics is the worst courier service I've ever used.",
    "ExpressWay Logistics failed to meet my expectations."
]

# Expected sentiments
expected_sentiments = ["Positive", "Positive", "Positive", "Negative", "Negative"]

# Evaluate zero-shot prompt
zero_shot_system_message = """
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""

# Get zero-shot results and F1 score
zero_shot_results = get_sentiment_classification(zero_shot_system_message)

# Print debug information for zero-shot results
print("Zero-Shot Prompt Results (Raw):")
print(zero_shot_results)

# Ensure we have exactly 5 results
zero_shot_results = zero_shot_results[:5]

print("Zero-Shot Prompt Results (Processed):")
print(zero_shot_results)

# Ensure the lengths match before calculating F1 score
if len(zero_shot_results) == len(expected_sentiments):
    zero_shot_f1 = f1_score(expected_sentiments, zero_shot_results, average='weighted')
    print("Zero-Shot Prompt Results:")
    print(zero_shot_results)
    print(f"Zero-Shot F1 Score: {zero_shot_f1}")
else:
    zero_shot_f1 = 0  # Handle case where results are not sufficient
    print("Zero-Shot results length mismatch. Skipping F1 score calculation.")

# Evaluate few-shot prompts and calculate F1 scores
few_shot_f1_scores = []

for examples in example_sets:
    few_shot_system_message = create_prompt(examples, reviews_to_classify)
    few_shot_results = get_sentiment_classification(few_shot_system_message)

    # Print debug information for few-shot results
    print("Few-Shot Prompt Results (Raw):")
    print(few_shot_results)

    # Ensure we have exactly 5 results
    few_shot_results = few_shot_results[:5]

    print("Few-Shot Prompt Results (Processed):")
    print(few_shot_results)

    if len(few_shot_results) == len(expected_sentiments):
        f1 = f1_score(expected_sentiments, few_shot_results, average='weighted')
        few_shot_f1_scores.append(f1)
        print(f"Few-Shot Prompt Results for example set: {examples}")
        print(f"Results: {few_shot_results}")
        print(f"F1 Score: {f1}")
    else:
        print(f"Skipping this set due to inconsistent result length: {few_shot_results}")

# Calculate mean and standard deviation of F1 scores
few_shot_mean = np.mean(few_shot_f1_scores)
few_shot_std = np.std(few_shot_f1_scores)

print("\nFew-Shot Evaluation Summary:")
print(f"Mean F1 Score: {few_shot_mean}")
print(f"Standard Deviation of F1 Scores: {few_shot_std}")

print("\nOverall Evaluation Summary:")
print(f"Zero-Shot F1 Score: {zero_shot_f1}")
print(f"Few-Shot Mean F1 Score: {few_shot_mean}")
print(f"Few-Shot Standard Deviation of F1 Scores: {few_shot_std}")


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Sending Prompt to OpenAI API:
 
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews:

1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]

API Response:
 {
  "id": "cmpl-9VyGy5ONGtA3BZhDSWNGN2KF5FAKd",
  "object": "text_completion",
  "created": 1717406884,
  "model": "gpt-35-turbo",
  "choices": [
    {


In [70]:
import json
import openai
import numpy as np
from sklearn.metrics import f1_score

# Define your configuration information
config_data = {
    "AZURE_OPENAI_KEY": "443225cb39c741c4a067057d418b9310",
    "AZURE_OPENAI_ENDPOINT": "https://openairesource02.openai.azure.com/",
    "AZURE_OPENAI_APITYPE": "azure",
    "AZURE_OPENAI_APIVERSION": "2023-03-15-preview",
    "CHATGPT_MODEL": "igpt-35-turbo"  # Ensure this is the correct model deployment name
}

# Write the configuration information into the config.json file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

print("Config file created successfully!")

# Load the configuration from config.json
try:
    with open('config.json') as config_file:
        config = json.load(config_file)
    print("Config file loaded successfully!")
except FileNotFoundError:
    print("Config file not found!")
    exit(1)
except json.JSONDecodeError:
    print("Error decoding the config file!")
    exit(1)

# Set up OpenAI API key and endpoint for Azure
openai.api_type = config['AZURE_OPENAI_APITYPE']
openai.api_base = config['AZURE_OPENAI_ENDPOINT']
openai.api_version = config['AZURE_OPENAI_APIVERSION']
openai.api_key = config['AZURE_OPENAI_KEY']

print("OpenAI API configuration set successfully!")

# Function to test a simple prompt
def simple_test():
    test_prompt = """
    Please classify the sentiment of the following review as either 'Positive' or 'Negative':
    "ExpressWay Logistics' commitment to transparency is impressive."
    """
    print("Sending Simple Test Prompt to OpenAI API:\n", test_prompt)
    try:
        response = openai.Completion.create(
            engine=config['CHATGPT_MODEL'],
            prompt=test_prompt,
            max_tokens=50,
            temperature=0.7
        )
        print("API Response:\n", response)
        return response.choices[0].text.strip()
    except openai.error.InvalidRequestError as e:
        print(f"InvalidRequestError: {e}")
    except openai.error.AuthenticationError as e:
        print(f"AuthenticationError: {e}")
    except openai.error.APIConnectionError as e:
        print(f"APIConnectionError: {e}")
    except openai.error.OpenAIError as e:
        print(f"OpenAIError: {e}")
    return ""

# Run the simple test
simple_test_result = simple_test()
print("Simple Test Result:", simple_test_result)

# Function to test a more complex prompt with examples
def complex_test():
    complex_prompt = """
I have a dataset of customer reviews. Each review has a sentiment associated with it. The sentiment can either be 'Positive' or 'Negative'.
Please classify the sentiment of the following reviews.

Examples:
1. "The best experience I've had in a long time." -> Positive
2. "The worst customer service I've encountered." -> Negative
3. "Excellent craftsmanship and attention to detail." -> Positive
4. "The product was defective and had to be returned." -> Negative
5. "Very happy with my purchase!" -> Positive
6. "I was disappointed with the overall experience." -> Negative

Please classify the following reviews:
1. "ExpressWay Logistics' commitment to transparency is impressive."
2. "The tracking system implemented by ExpressWay is user-friendly and reliable."
3. "ExpressWay Logistics is a lifesaver when it comes to timely deliveries."
4. "Expressway Logistics is the worst courier service I've ever used."
5. "ExpressWay Logistics failed to meet my expectations."

Please provide the sentiment for each review in the format:
1. [Sentiment]
2. [Sentiment]
3. [Sentiment]
4. [Sentiment]
5. [Sentiment]
"""
    print("Sending Complex Test Prompt to OpenAI API:\n", complex_prompt)
    try:
        response = openai.Completion.create(
            engine=config['CHATGPT_MODEL'],
            prompt=complex_prompt,
            max_tokens=50,
            temperature=0.7
        )
        print("API Response:\n", response)
        return response.choices[0].text.strip().split("\n")
    except openai.error.InvalidRequestError as e:
        print(f"InvalidRequestError: {e}")
    except openai.error.AuthenticationError as e:
        print(f"AuthenticationError: {e}")
    except openai.error.APIConnectionError as e:
        print(f"APIConnectionError: {e}")
    except openai.error.OpenAIError as e:
        print(f"OpenAIError: {e}")
    return []

# Run the complex test
complex_test_results = complex_test()
print("Complex Test Results (Raw):", complex_test_results)
complex_test_results = [line.split(" ")[-1].strip("[]") for line in complex_test_results if line]
print("Complex Test Results (Processed):", complex_test_results)


Config file created successfully!
Config file loaded successfully!
OpenAI API configuration set successfully!
Sending Simple Test Prompt to OpenAI API:
 
    Please classify the sentiment of the following review as either 'Positive' or 'Negative':
    "ExpressWay Logistics' commitment to transparency is impressive."
    
API Response:
 {
  "id": "cmpl-9VyMwWIp56W725P0B1F1cN9o75bN8",
  "object": "text_completion",
  "created": 1717407254,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "text": "\u00a0\n    Answer: Positive\n    \n    2.\n    Please classify the sentiment of the following review as either 'Positive' or 'Negative':\n    \"I was really disappointed with the quality of food at the local cafe.\"\n    \u00a0\n    Answer: Negative",
      "index": 0,
      "finish_reason": "length",
      "logprobs": null
    }
  ],
  "usage": {
    "prompt_tokens": 32,
    "completion_tokens": 50,
    "total_tokens": 82
  }
}
Simple Test Result: Answer: Positive
    
    2.
    Please c

##**Step 5: Observation and Insights and Business perspective (10 Marks)**

Based on the projects, learner needs to share observations, learnings, insights and the business use case where these learnings can be beneficial.
Provide a breakdown of the percentage of positive and negative reviews. Additionally, explain how this classification can assist ExpressWay Logistics in addressing the issues identified.


Observations and Insights
Zero-Shot vs. Few-Shot Performance:

Zero-shot prompts generally provide lower accuracy as the model is not given any specific examples to learn from.
Few-shot prompts improve accuracy by providing the model with examples, allowing it to better understand the context and nuances of the task.
Variability in Performance:

The performance (F1 score) of few-shot prompts can vary based on the examples provided.
Multiple runs with different examples help in understanding the variability and overall robustness of the model's performance.
Sentiment Distribution:

The breakdown of reviews into positive and negative sentiments allows for a clear understanding of customer satisfaction.
Business Use Case
Customer Sentiment Analysis for ExpressWay Logistics:

Objective: Understand customer sentiment to improve services and address specific issues highlighted by customers.
Benefit: By classifying customer reviews into positive and negative categories, ExpressWay Logistics can gain insights into customer satisfaction and dissatisfaction.
Actionable Insights:
Positive Feedback: Identify areas where the company excels and continue to reinforce these strengths.
Negative Feedback: Pinpoint specific issues or areas where customers are dissatisfied and prioritize them for improvement.
Breakdown of Sentiment
Here is a hypothetical breakdown based on the reviews used in the example:

Total Reviews: 10
Positive Reviews: 5
Negative Reviews: 5
Percentage Breakdown:
Positive Reviews: 50%
Negative Reviews: 50%
Business Perspective
How Sentiment Classification Can Assist ExpressWay Logistics:

Improving Customer Experience:

Identifying Pain Points: By analyzing negative reviews, ExpressWay Logistics can identify common issues such as delayed deliveries, damaged packaging, and poor customer service.
Proactive Measures: Implement corrective actions to address these issues, such as improving delivery processes, enhancing packaging standards, and training customer service staff.
Enhancing Positive Experiences:

Reinforcing Strengths: Positive reviews highlight what the company is doing well, such as timely deliveries and excellent customer service.
Customer Retention: Reinforce these strengths to maintain high levels of customer satisfaction and loyalty.
Data-Driven Decision Making:

Strategic Planning: Use sentiment analysis to guide strategic decisions, such as areas for investment, process improvements, and customer engagement strategies.
Resource Allocation: Allocate resources efficiently to address the most critical issues affecting customer satisfaction.
Conclusion
The implementation of sentiment analysis for customer reviews provides valuable insights that can significantly impact business decisions and customer satisfaction at ExpressWay Logistics. By leveraging these insights, the company can continuously improve its services, address customer concerns proactively, and maintain a competitive edge in the logistics industry. The process of evaluating both zero-shot and few-shot prompts also highlights the importance of context and examples in improving model performance, ultimately leading to more accurate and actionable insights.

**----------------------------------------------------------------------------End-----------------------------------------------------------------------------------------**